In [447]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import regex as re
import math
from collections import defaultdict

from rapidfuzz.fuzz import ratio

things to look at: how accurate is the prematch probability when it ends (if a team has over 50% chance to win, how often do they actually win the game?)

if my fair probability is wildely different than the midprice, should I weigh it less or find a new fair probability? How often is my direction right?

can i still find edge in a very wide market?

In [448]:
date = '2025-11-25'
odds_sport = 'cbb' #cbb, cfb, nba, nfl
kalshi_sport = 'ncaab' #ncaab, ncaaf, nba, nfl

In [449]:
odds_df = pd.read_csv(f"../data_collection/updated_scripts/oddsapi_outputs/{date}/{odds_sport}_odds.csv")
odds_df.drop(columns=['league'], inplace=True)
odds_df.rename(columns={'price': 'odds'}, inplace=True)
odds_df['vig_prob'] = 1 / odds_df['odds']

def remove_vig_probs(df):
    df = df.copy()
    df['fair_prb'] = pd.NA

    grouped = df.groupby(['game_id', 'bookmaker', 'market'])

    for _, group in grouped:
        if len(group) < 2:
            continue
        probs = group['vig_prob']
        total = probs.sum()
        if total == 0:
            continue
        fair_probs = (probs / total).round(4)
        df.loc[group.index, 'fair_prb'] = fair_probs

    return df

odds_df = remove_vig_probs(odds_df)


odds_winners_df = odds_df[odds_df['market'] == 'h2h'].copy()
odds_spreads_df = odds_df[odds_df['market'] == 'spreads'].copy()
odds_spreads_df = odds_spreads_df.loc[(odds_spreads_df['point'].notna()) & (odds_spreads_df['point'] > 0)]
odds_totals_df  = odds_df[odds_df['market'] == 'totals'].copy()

# Average per-team fair probabilities across DraftKings/FanDuel/Pinnacle for winners_df
mask = odds_winners_df['fair_prb'].notna()
avg_by_team = (
    odds_winners_df.loc[mask]
    .groupby(['game_id', 'team'])['fair_prb']
    .transform('median')
    .round(4)
)
odds_winners_df.loc[mask, 'avg_fair_prb'] = avg_by_team
odds_winners_df.loc[~mask, 'avg_fair_prb'] = pd.NA

#Average fair probabilities for spreads for same game, point spread, and team
mask = odds_spreads_df['fair_prb'].notna()
avg_by_point = (
    odds_spreads_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_spreads_df['avg_fair_prb'] = avg_by_point

#Average fair probabilities for totals for same game, point spread, direction (Over/Under)
mask = odds_totals_df['fair_prb'].notna()
avg_by_tot_point = (
    odds_totals_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_totals_df['avg_fair_prb'] = avg_by_tot_point

In [450]:
kalshi_winners_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_winners.csv")
kalshi_totals_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_totals.csv")
kalshi_spreads_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_spreads.csv")

if kalshi_sport == 'ncaaf':
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)
elif (kalshi_sport == 'ncaab') | (kalshi_sport == 'nba'):
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) Points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)
    



columns_to_drop = ['timestamp', 'market_type', 'yes_bid2', 'yes_ask2', 'no_bid2', 'no_ask2', 'yes_depth_bids', 'yes_depth_asks', 'no_depth_bids', 'no_depth_asks']
kalshi_winners_df.drop(columns=columns_to_drop, inplace=True)
kalshi_spreads_df.drop(columns=columns_to_drop, inplace=True)
kalshi_totals_df.drop(columns=columns_to_drop, inplace=True)

In [451]:
odds_totals_df.head()

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
4,CBB,f7941a2ca17e16dc2edfd7384e3eeef2,2025-11-25 10:00:00 CST,FanDuel,totals,Over,1.91,145.5,Bethune-Cookman Wildcats,Stony Brook Seawolves,0.523560,0.5,0.5
5,CBB,f7941a2ca17e16dc2edfd7384e3eeef2,2025-11-25 10:00:00 CST,FanDuel,totals,Under,1.91,145.5,Bethune-Cookman Wildcats,Stony Brook Seawolves,0.523560,0.5,0.5
10,CBB,f7941a2ca17e16dc2edfd7384e3eeef2,2025-11-25 10:00:00 CST,DraftKings,totals,Over,1.91,144.5,Bethune-Cookman Wildcats,Stony Brook Seawolves,0.523560,0.5,0.5035
11,CBB,f7941a2ca17e16dc2edfd7384e3eeef2,2025-11-25 10:00:00 CST,DraftKings,totals,Under,1.91,144.5,Bethune-Cookman Wildcats,Stony Brook Seawolves,0.523560,0.5,0.4965
16,CBB,f7941a2ca17e16dc2edfd7384e3eeef2,2025-11-25 10:00:00 CST,ESPN BET,totals,Over,1.87,144.5,Bethune-Cookman Wildcats,Stony Brook Seawolves,0.534759,0.5105,0.5035


In [452]:
#get names from kalshi_winners_df
def extract_teams_from_winners(title):
    title = title.replace(" Winner?", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
    elif " vs " in title:
        right, left = title.split(" vs ", 1)
    else:
        return pd.Series([None, None])  
    left = re.sub(r'\bSt\.$', 'St', left.strip())
    right = re.sub(r'\bSt\.$', 'St', right.strip())
    return pd.Series([left, right])

kalshi_winners_df[['home_team', 'away_team']] = kalshi_winners_df['title'].apply(extract_teams_from_winners)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_winners_teams = flat_teams.tolist()

#get names from kalshi_totals_df
def extract_teams_from_totals(title):
    title = title.replace(": Total Points", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
        left = re.sub(r'\bSt\.$', 'St', left.strip())
        right = re.sub(r'\bSt\.$', 'St', right.strip())
        return pd.Series([left, right])
    return None

kalshi_totals_df[['home_team', 'away_team']] = kalshi_totals_df['title'].apply(extract_teams_from_totals)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_totals_teams = flat_teams.tolist()

#get names from kalshi_spreads_df
def extract_team_from_spreads(title):
    if " wins by " in title:
        team = title.split(" wins by ", 1)[0].strip()
        team = re.sub(r'\bSt\.$', 'St', team)
        return team
    return None

kalshi_spreads_df['team'] = kalshi_spreads_df['title'].apply(extract_team_from_spreads)
unique_teams_spread = kalshi_spreads_df['team'].drop_duplicates()
kalshi_spreads_teams = unique_teams_spread.tolist()

In [453]:
odds_teams_by_market = odds_df.groupby('market')['team'].unique().to_dict()

def fuzzy_match_kalshi_to_odds(kalshi_teams, odds_team_names):
    matched_kalshi = []
    matched_odds = []
    candidates_dict = defaultdict(list)

    kalshi_sorted = sorted(kalshi_teams, key=lambda x: x[0] if x else '')
    remaining_odds = sorted(odds_team_names.tolist().copy())

    for kalshi_name in kalshi_sorted:
        candidates = []
        for odds_name in remaining_odds:
            if kalshi_name in odds_name:
                candidates.append(odds_name)
        if len(candidates) == 1:
            candidates_dict[candidates[0]].append(kalshi_name)
        elif len(candidates) > 1:
            best_fit = candidates[0]
            best_ratio = ratio(best_fit, kalshi_name)
            for name in candidates:
                curr_ratio = ratio(name, kalshi_name)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
            candidates_dict[best_fit].append(kalshi_name)
    
    for odd, kalsh in candidates_dict.items():
        best_fit = kalsh[0]
        best_ratio = ratio(best_fit, odd)
        if len(kalsh) > 1:
            for name in kalsh:
                curr_ratio = ratio(name, odd)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
        matched_odds.append(odd)
        matched_kalshi.append(best_fit)


    return matched_kalshi, matched_odds


# Winners / h2h
matched_kalshi_h2h, matched_odds_h2h = fuzzy_match_kalshi_to_odds(
    kalshi_winners_teams,
    odds_teams_by_market.get('h2h', [])
)

# Spreads
matched_kalshi_spreads, matched_odds_spreads = fuzzy_match_kalshi_to_odds(
    kalshi_spreads_teams,
    odds_teams_by_market.get('spreads', [])
)

# Totals (match only Over/Under)
totals_odds_df = odds_df[odds_df['market'] == 'totals']
odds_totals_teams = pd.unique(totals_odds_df[['home_team', 'away_team']].values.ravel())
matched_kalshi_totals, matched_odds_totals = fuzzy_match_kalshi_to_odds(
    kalshi_totals_teams,
    odds_totals_teams
)

matched_names = {
    'h2h': {
        'kalshi': matched_kalshi_h2h,
        'odds': matched_odds_h2h
    },
    'spreads': {
        'kalshi': matched_kalshi_spreads,
        'odds': matched_odds_spreads
    },
    'totals': {
        'kalshi': matched_kalshi_totals,
        'odds': matched_odds_totals
    }
}


In [454]:
assert(len(matched_names['h2h']['kalshi']) == len(matched_names['h2h']['odds']))
assert(len(matched_names['spreads']['kalshi']) == len(matched_names['spreads']['odds']))
assert(len(matched_names['totals']['kalshi']) == len(matched_names['totals']['odds']))

In [455]:
odds_winners_df = odds_winners_df[
    odds_winners_df['home_team'].isin(matched_names['h2h']['odds']) |
    odds_winners_df['away_team'].isin(matched_names['h2h']['odds'])
].drop_duplicates(subset='team').sort_values(by='home_team').reset_index(drop=True)

kalshi_winners_df = kalshi_winners_df[
    kalshi_winners_df['home_team'].isin(matched_names['h2h']['kalshi']) |
    kalshi_winners_df['away_team'].isin(matched_names['h2h']['kalshi'])
].sort_values(by='home_team').reset_index(drop=True)

odds_spreads_df = odds_spreads_df[odds_spreads_df['team'].isin(matched_names['spreads']['odds'])].reset_index(drop=True)
kalshi_spreads_df = kalshi_spreads_df[kalshi_spreads_df['team'].isin(matched_names['spreads']['kalshi'])].reset_index(drop=True)

odds_totals_df = odds_totals_df[
    odds_totals_df['home_team'].isin(matched_names['totals']['odds']) |
    odds_totals_df['away_team'].isin(matched_names['totals']['odds'])
].sort_values(by='home_team').reset_index(drop=True)
kalshi_totals_df = kalshi_totals_df[
    (kalshi_totals_df['home_team'].isin(matched_names['totals']['kalshi'])) | 
    (kalshi_totals_df['away_team'].isin(matched_names['totals']['kalshi']))
    ].sort_values(by='home_team').reset_index(drop=True)


In [456]:
# Concatenate winners df

# Specify the columns to extract
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'home_team', 'away_team']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb']

# Rename overlapping columns in odds to prevent clashes
odds_subset = odds_winners_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team'
})

kalshi_subset = kalshi_winners_df[kalshi_cols].rename(columns={
    'home_team': 'kalshi_home_team',
    'away_team': 'kalshi_away_team'
})

combined_rows = []
len_matched = matched_names['h2h']['kalshi']
matched_names_h2h = matched_names['h2h']

for i in range(len(len_matched)):
    odds_name = matched_names_h2h['odds'][i]
    kalshi_name = matched_names_h2h['kalshi'][i]

    # Find the corresponding odds row
    odds_row = odds_subset.loc[odds_subset['team'] == odds_name]
    assert len(odds_row) == 1, f"Expected one row for {odds_name}, got {len(odds_row)}"

    # Find the two matching Kalshi rows
    kalshi_rows = kalshi_subset.loc[
        (kalshi_subset['kalshi_home_team'] == kalshi_name) |
        (kalshi_subset['kalshi_away_team'] == kalshi_name)
    ]
    assert len(kalshi_rows) == 2, f"Expected two rows for {kalshi_name}, got {len(kalshi_rows)}"

    # Extract rows
    k1 = kalshi_rows.iloc[0]
    k2 = kalshi_rows.iloc[1]
    midprice1 = (k1['yes_bid'] + k1['yes_ask']) / 2
    midprice2 = (k2['yes_bid'] + k2['yes_ask']) / 2

    # Extract scalar fair probability
    prb = odds_row['avg_fair_prb'].astype(float).item()

    # Choose the row closer to the odds probability
    if ((midprice1 - prb) ** 2) < ((midprice2 - prb) ** 2):
        combined_row = pd.concat([k1, odds_row.iloc[0]])
    else:
        combined_row = pd.concat([k2, odds_row.iloc[0]])

    combined_rows.append(combined_row)

combined_winners_df = pd.DataFrame(combined_rows)
combined_winners_df = combined_winners_df.reset_index(drop=True)

In [457]:
EDGE = 0.00
KELLY_UPPERBOUND = 1
BANKROLL = 337.00
Q1_WEIGHT = 1.00
Q2_WEIGHT = 1.00
Q3_WEIGHT = 1.00
Q4_WEIGHT = 1.00

edge_winners_df = combined_winners_df.loc[
    (combined_winners_df['avg_fair_prb'] >= combined_winners_df['yes_ask'] + EDGE) |
    (combined_winners_df['avg_fair_prb'] <= combined_winners_df['yes_bid'] - EDGE)
].reset_index(drop=True)

midprice = (edge_winners_df['yes_bid'] + edge_winners_df['yes_ask']) / 2

q = edge_winners_df['avg_fair_prb']
p = midprice

edge_winners_df['raw_kelly'] = np.where(
    q > p,
    (q - p) / (1 - p),      # long position
    (p - q) / p           # short position
)

total_kelly = edge_winners_df['raw_kelly'].sum()
edge_winners_df['raw_kelly'] = pd.DataFrame({
    'original': edge_winners_df['raw_kelly'],
    'normalized': (edge_winners_df['raw_kelly'] / total_kelly)
}).min(axis=1)

# Define the real_kelly logic
def scale_kelly(row):
    k = row['raw_kelly']
    p = row['avg_fair_prb']
    
    if k == 0 or pd.isna(k):
        return 0
    if 0.05 <= p < 0.25:
        return min(Q1_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.25 <= p < 0.5:
        return min(Q2_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.5 <= p < 0.75:
        return min(Q3_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.75 <= p < 0.95:
        return min(Q4_WEIGHT * k, KELLY_UPPERBOUND)
    else:
        return 0  # fallback if out of range

# Apply to the DataFrame
edge_winners_df['real_kelly'] = edge_winners_df.apply(scale_kelly, axis=1)
edge_winners_df['optimal_bet'] = edge_winners_df['real_kelly'] * BANKROLL

num_contracts = edge_winners_df['optimal_bet'] // edge_winners_df['yes_bid']
edge_winners_df['num_contracts'] = num_contracts
trading_cost = np.ceil(100*(0.0175 * num_contracts * edge_winners_df['yes_bid'] * (1 - edge_winners_df['yes_bid']))) / 100
edge_winners_df['trading_cost'] = trading_cost
profit = (1 - edge_winners_df['yes_bid']) *  num_contracts - trading_cost
edge_winners_df['profit'] = profit
edge_winners_df['ev'] = (profit * edge_winners_df['avg_fair_prb'] - edge_winners_df['optimal_bet'] * (1 - edge_winners_df['avg_fair_prb'])).round(2)
filtered_winners_df = edge_winners_df.loc[edge_winners_df['ev'] > 0.15].reset_index(drop=True)

In [458]:
filtered_winners_df

,ticker,yes_bid,yes_ask,kalshi_home_team,kalshi_away_team,market,start_time,team,odds_home_team,odds_away_team,avg_fair_prb,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev
0,KXNCAAMBGAME-25NOV25NCSTBSU-BSU,0.26,0.28,Boise St,North Carolina St,h2h,2025-11-25 13:30:17 CST,Boise State Broncos,Boise State Broncos,NC State Wolfpack,0.28955,0.012622,0.012622,4.253539,16.0,0.06,11.78,0.39
1,KXNCAAMBGAME-25NOV25UCLACAL-CAL,0.22,0.23,California,UCLA,h2h,2025-11-25 21:00:00 CST,California Golden Bears,California Golden Bears,UCLA Bruins,0.23950,0.008818,0.008818,2.971617,13.0,0.04,10.10,0.16
2,KXNCAAMBGAME-25NOV25CAMPWAKE-CAMP,0.05,0.06,Wake Forest,Campbell,h2h,2025-11-25 18:00:00 CST,Campbell Fighting Camels,Wake Forest Demon Deacons,Campbell Fighting Camels,0.07695,0.010947,0.010947,3.689174,73.0,0.07,69.28,1.93
3,KXNCAAMBGAME-25NOV25ISUCREI-CREI,0.23,0.25,Creighton,Iowa St,h2h,2025-11-25 13:00:00 CST,Creighton Bluejays,Creighton Bluejays,Iowa State Cyclones,0.25850,0.011472,0.011472,3.866203,16.0,0.05,12.27,0.31
4,KXNCAAMBGAME-25NOV25EWUUNT-EWU,0.11,0.13,North Texas,Eastern Washington,h2h,2025-11-25 19:00:00 CST,Eastern Washington Eagles,North Texas Mean Green,Eastern Washington Eagles,0.13690,0.009051,0.009051,3.050215,27.0,0.05,23.98,0.65
5,KXNCAAMBGAME-25NOV25MISSIOWA-IOWA,0.62,0.63,Iowa,Ole Miss,h2h,2025-11-25 20:30:00 CST,Iowa Hawkeyes,Iowa Hawkeyes,Ole Miss Rebels,0.64590,0.026267,0.026267,8.852002,14.0,0.06,5.26,0.26
6,KXNCAAMBGAME-25NOV25KSUIND-KSU,0.18,0.20,Indiana,Kansas St,h2h,2025-11-25 19:00:00 CST,Kansas St Wildcats,Indiana Hoosiers,Kansas St Wildcats,0.20690,0.009833,0.009833,3.313814,18.0,0.05,14.71,0.42
7,KXNCAAMBGAME-25NOV25ODUVILL-ODU,0.03,0.05,Villanova,Old Dominion,h2h,2025-11-25 18:00:00 CST,Old Dominion Monarchs,Villanova Wildcats,Old Dominion Monarchs,0.05855,0.009107,0.009107,3.069016,102.0,0.06,98.88,2.90
8,KXNCAAMBGAME-25NOV25ORESDSU-SDSU,0.42,0.43,San Diego St,Oregon,h2h,2025-11-25 22:00:00 CST,San Diego St Aztecs,San Diego St Aztecs,Oregon Ducks,0.44515,0.016516,0.016516,5.565878,13.0,0.06,7.48,0.24
9,KXNCAAMBGAME-25NOV25SBONUNC-SBON,0.12,0.14,North Carolina,St. Bonaventure,h2h,2025-11-25 17:00:00 CST,St. Bonaventure Bonnies,North Carolina Tar Heels,St. Bonaventure Bonnies,0.15970,0.016089,0.016089,5.422052,45.0,0.09,39.51,1.75


In [459]:
total_loss = np.sum(filtered_winners_df['optimal_bet'])
total_profit = np.sum(filtered_winners_df['profit'])
total_ev = np.sum(filtered_winners_df['ev'])
print(f"{odds_sport} h2h portfolio summary:\n")
print(f"Max Loss: -{total_loss:.2f}")
print(f"Max Profit: {total_profit:.2f}")
print(f"Portfolio EV: {total_ev:.2f}")


cbb h2h portfolio summary:

Max Loss: -52.17
Max Profit: 339.09
Portfolio EV: 10.12


In [460]:
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'team', 'points']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb', 'point']

odds_subset = odds_spreads_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team',
    'team': 'odds_team'
})

kalshi_subset = kalshi_spreads_df[kalshi_cols]

combined_rows = []

for _, kalshi_row in kalshi_subset.iterrows():
    kalshi_home = kalshi_row['team']
    for _, odds_row in odds_subset.iterrows():
        odds_home = odds_row['odds_team']
        if (kalshi_home in odds_home) and (kalshi_row['points'] == odds_row['point']): 
            combined_row = pd.concat([kalshi_row, odds_row])
            combined_rows.append(combined_row)

combined_spreads_df = pd.DataFrame(combined_rows).drop_duplicates(subset='ticker') #only works because oddsapi only pulls odds 
combined_spreads_df = combined_spreads_df.reset_index(drop=True)                    #for only one point line for each bookmaker 



In [461]:
combined_spreads_df

,ticker,yes_bid,yes_ask,team,points,market,start_time,odds_team,odds_home_team,odds_away_team,avg_fair_prb,point
0,KXNCAAMBSPREAD-25NOV25UTSAGASO-UTSA2,0.36,0.40,UTSA,2.5,spreads,2025-11-25 20:00:00 CST,UTSA Roadrunners,Georgia Southern Eagles,UTSA Roadrunners,0.500000,2.5
1,KXNCAAMBSPREAD-25NOV25FAUGMU-FAU5,0.15,0.23,Florida Atlantic,5.5,spreads,2025-11-25 18:30:38 CST,Florida Atlantic Owls,Florida Atlantic Owls,George Mason Patriots,0.504980,5.5
2,KXNCAAMBSPREAD-25NOV25TEMPRIN-PRIN8,0.03,0.52,Princeton,8.5,spreads,2025-11-25 18:30:55 CST,Princeton Tigers,Princeton Tigers,Temple Owls,0.494750,8.5
3,KXNCAAMBSPREAD-25NOV25UTEPUAB-UTEP8,0.03,0.52,UTEP,8.5,spreads,2025-11-25 15:00:00 CST,UTEP Miners,UAB Blazers,UTEP Miners,0.512200,8.5
4,KXNCAAMBSPREAD-25NOV25NCSTBSU-BSU6,0.03,0.97,Boise St,6.5,spreads,2025-11-25 13:30:17 CST,Boise State Broncos,Boise State Broncos,NC State Wolfpack,0.511800,6.5
5,KXNCAAMBSPREAD-25NOV25JACPAC-JAC4,0.17,0.43,Jacksonville,4.5,spreads,2025-11-25 12:30:00 CST,Jacksonville Dolphins,Jacksonville Dolphins,Pacific Tigers,0.506500,4.5
6,KXNCAAMBSPREAD-25NOV25MISSIOWA-MISS4,0.20,0.39,Ole Miss,4.5,spreads,2025-11-25 20:30:00 CST,Ole Miss Rebels,Iowa Hawkeyes,Ole Miss Rebels,0.496620,4.5
7,KXNCAAMBSPREAD-25NOV25ECUMSU-ECU22,0.03,0.95,East Carolina,22.5,spreads,2025-11-25 12:00:00 CST,East Carolina Pirates,Michigan St Spartans,East Carolina Pirates,0.509167,22.5
8,KXNCAAMBSPREAD-25NOV25ISUCREI-CREI6,0.03,0.97,Creighton,6.5,spreads,2025-11-25 13:00:00 CST,Creighton Bluejays,Creighton Bluejays,Iowa State Cyclones,0.487900,6.5
9,KXNCAAMBSPREAD-25NOV25NDRUTG-RUTG6,0.03,0.40,Rutgers,6.5,spreads,2025-11-25 12:00:00 CST,Rutgers Scarlet Knights,Rutgers Scarlet Knights,Notre Dame Fighting Irish,0.520950,6.5


In [462]:
EDGE = 0.01
KELLY_UPPERBOUND = 1
BANKROLL = 35.00
Q1_WEIGHT = 1.00
Q2_WEIGHT = 1.00
Q3_WEIGHT = 1.00
Q4_WEIGHT = 1.00

edge_spreads_df = combined_spreads_df.loc[(combined_spreads_df['avg_fair_prb'] >= combined_spreads_df['yes_ask'] + EDGE) |
                        (combined_spreads_df['avg_fair_prb'] <= combined_spreads_df['yes_bid'] - EDGE)].reset_index(drop=True)

midprice = (edge_spreads_df['yes_bid'] + edge_spreads_df['yes_ask']) / 2

q = edge_spreads_df['avg_fair_prb']
p = midprice

edge_spreads_df['raw_kelly'] = np.where(
    q > p,
    (q - p) / (1 - p),      # long position
    (p - q) / p           # short position
)

total_kelly = edge_spreads_df['raw_kelly'].sum()
edge_spreads_df['raw_kelly'] = pd.DataFrame({
    'original': edge_spreads_df['raw_kelly'],
    'normalized': (edge_spreads_df['raw_kelly'] / total_kelly)
}).min(axis=1)

# Define the real_kelly logic
def scale_kelly(row):
    k = row['raw_kelly']
    p = row['avg_fair_prb']
    
    if k == 0 or pd.isna(k):
        return 0
    if 0.1 <= p < 0.25:
        return min(Q1_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.25 <= p < 0.5:
        return min(Q2_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.5 <= p < 0.75:
        return min(Q3_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.75 <= p < 0.9:
        return min(Q4_WEIGHT * k, KELLY_UPPERBOUND)
    else:
        return 0  # fallback if out of range

# Apply to the DataFrame
edge_spreads_df['real_kelly'] = edge_spreads_df.apply(scale_kelly, axis=1)
edge_spreads_df['optimal_bet'] = edge_spreads_df['real_kelly'] * BANKROLL

num_contracts = edge_spreads_df['optimal_bet'] // edge_spreads_df['yes_bid']
edge_spreads_df['num_contracts'] = num_contracts
trading_cost = np.ceil(100 * (0.0175 * num_contracts * edge_spreads_df['yes_bid'] * (1 - edge_spreads_df['yes_bid']))) / 100
edge_spreads_df['trading_cost'] = trading_cost
profit = (1 - edge_spreads_df['yes_bid']) *  num_contracts - trading_cost
edge_spreads_df['profit'] = profit
edge_spreads_df['ev'] = (profit * edge_spreads_df['avg_fair_prb'] - edge_spreads_df['optimal_bet'] * (1 - edge_spreads_df['avg_fair_prb'])).round(2)
filtered_spreads_df = edge_spreads_df.loc[edge_spreads_df['ev'] > 0].reset_index(drop=True)

In [463]:
filtered_spreads_df

,ticker,yes_bid,yes_ask,team,points,market,start_time,odds_team,odds_home_team,odds_away_team,avg_fair_prb,point,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev
0,KXNCAAMBSPREAD-25NOV25UTSAGASO-UTSA2,0.36,0.40,UTSA,2.5,spreads,2025-11-25 20:00:00 CST,UTSA Roadrunners,Georgia Southern Eagles,UTSA Roadrunners,0.50000,2.5,0.124617,0.124617,4.361601,12.0,0.05,7.63,1.63
1,KXNCAAMBSPREAD-25NOV25FAUGMU-FAU5,0.15,0.23,Florida Atlantic,5.5,spreads,2025-11-25 18:30:38 CST,Florida Atlantic Owls,Florida Atlantic Owls,George Mason Patriots,0.50498,5.5,0.250372,0.250372,8.763032,58.0,0.13,49.17,20.49
2,KXNCAAMBSPREAD-25NOV25JACPAC-JAC4,0.17,0.43,Jacksonville,4.5,spreads,2025-11-25 12:30:00 CST,Jacksonville Dolphins,Jacksonville Dolphins,Pacific Tigers,0.50650,4.5,0.189937,0.189937,6.647808,39.0,0.10,32.27,13.06
3,KXNCAAMBSPREAD-25NOV25MISSIOWA-MISS4,0.20,0.39,Ole Miss,4.5,spreads,2025-11-25 20:30:00 CST,Ole Miss Rebels,Iowa Hawkeyes,Ole Miss Rebels,0.49662,4.5,0.184134,0.184134,6.444673,32.0,0.09,25.51,9.42
4,KXNCAAMBSPREAD-25NOV25NDRUTG-RUTG6,0.03,0.40,Rutgers,6.5,spreads,2025-11-25 12:00:00 CST,Rutgers Scarlet Knights,Rutgers Scarlet Knights,Notre Dame Fighting Irish,0.52095,6.5,0.250940,0.250940,8.782886,292.0,0.15,283.09,143.27


In [464]:
total_loss = np.sum(filtered_spreads_df['optimal_bet'])
total_profit = np.sum(filtered_spreads_df['profit'])
total_ev = np.sum(filtered_spreads_df['ev'])
print(f"{odds_sport} spreads portfolio summary:\n")
print(f"Max Loss: -{total_loss:.2f}")
print(f"Max Profit: {total_profit:.2f}")
print(f"Portfolio EV: {total_ev:.2f}")


cbb spreads portfolio summary:

Max Loss: -35.00
Max Profit: 397.67
Portfolio EV: 187.87
